## Xarray

Converting the NetCDF4 file that stores the information for the machine-learning Human Footprint Index (ml-HFI) results into a DataFrame for inspection and analysis.

Data from: 
Keys, P. W., Barnes, E. A., & Carter, N. Dataset associated with "A machine-learning approach to human footprint index estimation with applications to sustainable development." Colorado State University. Libraries. http://dx.doi.org/10.25675/10217/216207

Method inspiration from https://stackoverflow.com/questions/66169106/transform-part-of-a-netcdf-file-into-a-dataframe-with-xarray:

In [ ]:
# necessary installations to run this notebook to be aware of: xarray, dask, bottleneck, rasterio

In [1]:
import pandas as pd
import numpy as np
import xarray as xr 
import rioxarray as rio 
import rasterio

In [2]:
dp = xr.open_dataset('./data/ml_hfi_v1_2000.nc')

ml = dp.to_dataframe()
ml = ml.dropna().reset_index()
ml.head()

         lat        lon  __xarray_dataarray_variable__
0 -55.609663 -68.108226                   3.328762e-06
1 -55.609663 -68.098333                   1.328018e-08
2 -55.609663 -68.088440                   1.276196e-13
3 -55.609663 -68.078547                   7.129802e-14
4 -55.609663 -68.068655                   6.915253e-18


In [3]:
ml.tail()

,lat,lon,__xarray_dataarray_variable__
140472840,69.988495,171.830159,1.399456e-07
140472841,69.988495,171.840051,2.790310e-04
140472842,69.988495,171.849944,3.740434e-05
140472843,69.988495,171.859837,4.128010e-02
140472844,69.988495,171.869730,8.492768e-04


### Created desired subset for AOI:
(40,30N & -110,-100W) which aligns with the Hansen Global Forest Cover tile that I've downloaded.

In [4]:
# Using the 2019 ml-HFI predictions: 
dp = xr.open_dataset('./data/ml_hfi_v1_2019.nc')

In [5]:
dp

<xarray.Dataset>
Dimensions:                        (lat: 13141, lon: 36390)
Coordinates:
  * lat                            (lat) float64 -59.99 -59.98 ... 69.99 70.0
  * lon                            (lon) float64 -180.0 -180.0 ... 180.0 180.0
Data variables:
    __xarray_dataarray_variable__  (lat, lon) float64 ...

In [6]:
dp['__xarray_dataarray_variable__']

<xarray.DataArray '__xarray_dataarray_variable__' (lat: 13141, lon: 36390)>
[478200990 values with dtype=float64]
Coordinates:
  * lat      (lat) float64 -59.99 -59.98 -59.97 -59.96 ... 69.98 69.99 70.0
  * lon      (lon) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0

In [7]:
# From https://stackoverflow.com/questions/29135885/netcdf4-extract-for-subset-of-lat-lon

ds = xr.open_dataset('./data/ml_hfi_v1_2019.nc')
lat_bnds, lon_bnds = [30, 40], [-110, -100]
subset = ds.sel(lat=slice(*lat_bnds), lon=slice(*lon_bnds))
subset

<xarray.Dataset>
Dimensions:                        (lat: 1011, lon: 1011)
Coordinates:
  * lat                            (lat) float64 30.0 30.01 ... 39.98 39.99
  * lon                            (lon) float64 -110.0 -110.0 ... -100.0 -100.0
Data variables:
    __xarray_dataarray_variable__  (lat, lon) float64 ...

In [8]:
type(subset)

xarray.core.dataset.Dataset

In [9]:
# Saving dataarray to netCDF
subset.to_netcdf('./data/ml_hfi_subset.nc')

### Convert .nc to geoTIFF
from: https://help.marine.copernicus.eu/en/articles/5029956-how-to-convert-netcdf-to-geotiff

In [10]:
nc_file = xr.open_dataset('./data/ml_hfi_subset.nc')
nc_file

<xarray.Dataset>
Dimensions:                        (lat: 1011, lon: 1011)
Coordinates:
  * lat                            (lat) float64 30.0 30.01 ... 39.98 39.99
  * lon                            (lon) float64 -110.0 -110.0 ... -100.0 -100.0
Data variables:
    __xarray_dataarray_variable__  (lat, lon) float64 ...

In [11]:
ml_hfi = nc_file['__xarray_dataarray_variable__']

In [12]:
ml_hfi = ml_hfi.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
ml_hfi.rio.crs

In [13]:
# Define the CRS projection
ml_hfi.rio.write_crs("epsg:4326", inplace=True)

<xarray.DataArray '__xarray_dataarray_variable__' (lat: 1011, lon: 1011)>
[1022121 values with dtype=float64]
Coordinates:
  * lat          (lat) float64 30.0 30.01 30.02 30.03 ... 39.97 39.98 39.99
  * lon          (lon) float64 -110.0 -110.0 -110.0 ... -100.0 -100.0 -100.0
    spatial_ref  int64 0

In [14]:
ml_hfi.rio.to_raster(r"./data/ml_hfi2019.tiff")

In [15]:
# Confirming that saving to raster was successful:

image_file = "./data/ml_hfi2019.tiff"

mlhfi_image = rasterio.open(image_file)

In [16]:
mlhfi_image

<open DatasetReader name='./data/ml_hfi2019.tiff' mode='r'>